# Modelo Multidimensional: Cube

## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

## Introdução

O presente notebook demonstra a utilização da funcionalidade [Cube](https://www.postgresql.org/docs/current/cube.html) para exploração de um banco de dados construido com PostgreSQL sobre os dados do projeto [FAPESP COVID-19 DataSharing](https://repositoriodatasharingfapesp.uspdigital.usp.br/). Conforme enuncido para este exercício.

### Prérequisitos

Assume-se

- a instalação de todas as dependências do projeto;
- a existência de um banco de dados pré-configurado conforme explicitado em [notebook anterior](https://github.com/de-abreu/data-mining/blob/main/01-Prepara%C3%A7%C3%A3o%20de%20dados/Gera%C3%A7%C3%A3o%20de%20Databases%20com%20SQLalchemy%20e%20PostgreSQL.ipynb);
- A correta configuração do acesso ao banco de dados abaixo.

## Configuração

- DATABASE: O nome do database onde serão carregadas as informações. Atente-se se este não corresponde ao nome de um database preexistente ou que esteja sendo acessado, pois este será então sobrescrito.

- USER e PASSWORD: Informações de autententicação válidas e com privilégios para a criação de bancos de dados no servidor.

- HOST e PORT: A URL e porta para realização do acesso ao servidor.


In [1]:
DATABASE = "fapcov2103"
USER = "postgres"
PASSWORD = "password"
HOST = "localhost"
PORT = 5432

DATABASE_URI = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

## Importação das dependências

In [21]:
import pandas as pd
from sqlalchemy import create_engine, select, func, text, MetaData, Table
from sqlalchemy.orm import declarative_base

## Enunciado

Listar a contagem de quantos Exames de Coronavírus foram feitos (procurar por ’corona’ ou ’covid’ em qualquer caixa de letra) por:

- Tipo de Exame
- Hospital em que o exame foi realizado
- Cidade de origem do paciente
- Quaisquer combinações dos fatores anteriores.

## Resolução

Todos os exames relacionados ao diagnóstico de Covid já foram separados na tabela AnalisesCovid do schema D2 no exercício anterior, então selecionamos esta e realizados a contagem dos subtotais e total usando Cube, como visto abaixo.

In [24]:
engine = create_engine(
    DATABASE_URI,
    connect_args={'options': f'-c search_path=D2'},
)

with engine.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS unaccent;"))
    conn.commit()

Base = declarative_base()
metadata = MetaData()

class ExamLab(Base):
    __table__ = Table("ExamLabs", metadata, autoload_with=engine)

class Paciente(Base):
    __table__ = Table("Pacientes", metadata, autoload_with=engine)

def normalize(column):
    return func.lower(func.trim(func.unaccent(column)))

exame=normalize(ExamLab.DE_Exame)

stmt = select(
    func.coalesce(exame, 'Todos').label("Tipo de Exame"),
    func.coalesce(func.trim(Paciente.CD_Municipio), 'Todos').label("Cidade do Paciente"),
    func.coalesce(func.trim(Paciente.DE_Hospital), 'Todos').label("Hospital"),
    func.count().label("Numero de Exames")
).select_from(
    ExamLab.__table__.join(Paciente.__table__, ExamLab.ID_Paciente == Paciente.ID_Paciente)
).where(
    # Use the .op() method for the database-specific regex operator (~* for PostgreSQL)
    exame.op('~*')(r'(covid)|(sars.cov.2)|(corona)')
).group_by(
    func.cube(
        exame,
        func.trim(Paciente.CD_Municipio),
        func.trim(Paciente.DE_Hospital)
    )
).order_by(
    "Tipo de Exame",
    "Cidade do Paciente",
    "Hospital"
)
df = pd.read_sql(stmt, engine)
with pd.option_context('display.max_rows', None):
    display(df.style.hide(axis="index"))

In [27]:
# The SQL query is now just a simple SELECT and JOIN
raw_data_stmt = select(
    ExamLab.DE_Exame,
    Paciente.CD_Municipio,
    Paciente.DE_Hospital
).select_from(
    ExamLab.__table__.join(Paciente.__table__, ExamLab.ID_Paciente == Paciente.ID_Paciente)
).where(
    # You can still apply pre-filters
    func.unaccent(ExamLab.DE_Exame).op('~*')(r'(covid)|(sars.cov.2)|(corona)')
)

# Fetch the result into a DataFrame
df_raw = pd.read_sql(raw_data_stmt, engine)

# (Optional but recommended) Clean the data in Pandas
df_raw['DE_Exame'] = df_raw['DE_Exame'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.strip().str.lower()
df_raw['CD_Municipio'] = df_raw['CD_Municipio'].str.strip()
df_raw['DE_hospital'] = df_raw['DE_Hospital'].str.strip()

In [30]:
df_cube = pd.pivot_table(
    df_raw,
    index=['DE_Hospital', 'CD_Municipio'],
    columns='DE_Exame',
    aggfunc=len,
    margins=True,
    margins_name='Todos',
    fill_value=0
)
with pd.option_context('display.max_rows', None):
    display(df_cube)

DE_hospital  \
DE_Exame                            anticorpos iga e igg sars-cov-2/covid19   
DE_Hospital CD_Municipio                                                      
BPSP        BARUERI                                                       0   
            CARAPICUIBA                                                   0   
            COTIA                                                         0   
            DIADEMA                                                       0   
            GUARULHOS                                                     0   
            MANAUS                                                        0   
            MOGI DAS CRUZES                                               0   
            OSASCO                                                        0   
            SANTANA DE PARNAIBA                                           0   
            SANTO ANDRE                                                   0   
            SANTOS                                                        0   
            SAO BERNARDO DO CAMPO                                         0   
            SAO PAULO                                                     0   
            SUZANO                                                        0   
Einstein    BARUERI                                                      28   
            CABREUVA                                                      0   
            CAMPINAS                                                      0   
            CARAPICUIBA                                                   0   
            COTIA                                                        14   
            DIADEMA                                                       0   
            EMBU                                                          2   
            FRANCISCO MORATO                                              0   
            FRANCO DA ROCHA                                               0   
            GUARULHOS                                                     4   
            ITAPECERICA DA SERRA                                          0   
            ITAPEVI                                                       0   
            JANDIRA                                                       0   
            JARINU                                                        0   
            JUNDIAI                                                       6   
            MAUA                                                          0   
            OSASCO                                                       12   
            PRAIA GRANDE                                                  0   
            SANTANA DE PARNAIBA                                          15   
            SANTO ANDRE                                                   2   
            SANTOS                                                        0   
            SAO BERNARDO DO CAMPO                                         0   
            SAO CAETANO DO SUL                                            0   
            SAO JOSE DOS CAMPOS                                           0   
            SAO PAULO                                                   532   
            SOROCABA                                                      0   
            TABOAO DA SERRA                                               0   
GrupoFleury ABREU E LIMA                                                  0   
            ALMIRANTE TAMANDARE                                           0   
            ALVORADA                                                      0   
            AMERICANA                                                     0   
            ARAUCARIA                                                     0   
            ARUJA                                                         0   
            ATIBAIA                                                       0   
            BARUERI                